In [1]:
from google.cloud import bigquery

In [2]:
client = bigquery.Client()

dataset_ref = client.dataset("nhtsa_traffic_fatalities", project="bigquery-public-data")
dataset = client.get_dataset(dataset_ref)

table_ref = dataset_ref.table("accident_2015")
table = client.get_table(table_ref)

client.list_rows(table, max_results=5).to_dataframe()

,state_number,state_name,consecutive_number,number_of_vehicle_forms_submitted_all,number_of_motor_vehicles_in_transport_mvit,number_of_parked_working_vehicles,number_of_forms_submitted_for_persons_not_in_motor_vehicles,number_of_persons_not_in_motor_vehicles_in_transport_mvit,number_of_persons_in_motor_vehicles_in_transport_mvit,number_of_forms_submitted_for_persons_in_motor_vehicles,...,minute_of_ems_arrival_at_hospital,related_factors_crash_level_1,related_factors_crash_level_1_name,related_factors_crash_level_2,related_factors_crash_level_2_name,related_factors_crash_level_3,related_factors_crash_level_3_name,number_of_fatalities,number_of_drunk_drivers,timestamp_of_crash
0,19,Iowa,190204,1,1,0,0,0,1,1,...,2,0,None,0,None,0,None,1,1,2015-09-11 20:20:00+00:00
1,19,Iowa,190233,1,1,0,0,0,1,1,...,88,0,None,0,None,0,None,1,1,2015-11-01 00:30:00+00:00
2,19,Iowa,190179,1,1,0,0,0,2,2,...,1,0,None,0,None,0,None,1,0,2015-05-04 16:18:00+00:00
3,19,Iowa,190248,1,1,0,0,0,4,4,...,99,0,None,0,None,0,None,2,0,2015-11-17 12:26:00+00:00
4,19,Iowa,190231,1,1,0,0,0,1,1,...,88,0,None,0,None,0,None,1,0,2015-10-31 04:49:00+00:00


In [3]:
#table.schema

In [4]:
safe_config = bigquery.QueryJobConfig(max_bytes_billed=10**10)

def get_query(my_query):
    query_job = client.query(my_query, job_config=safe_config)
    accidents_by_day = query_job.to_dataframe()
    return accidents_by_day

In [5]:
query1 = """
        SELECT COUNT(consecutive_number) AS num_accidents,
            EXTRACT(DAYOFWEEK FROM timestamp_of_crash) AS day_of_week
        FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015`
        GROUP BY day_of_week
        ORDER BY num_accidents DESC
        """

In [6]:
d1 = get_query(query1)
d1

,num_accidents,day_of_week
0,5659,7
1,5298,1
2,4916,6
3,4460,5
4,4182,4
5,4038,2
6,3985,3


In [7]:
query2 = """
        SELECT COUNT(1) AS num_accidents,
            EXTRACT(DAYOFWEEK FROM timestamp_of_crash) AS day_of_week
        FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015`
        GROUP BY day_of_week
        ORDER BY num_accidents DESC
        """

d2 = get_query(query2)
d2

,num_accidents,day_of_week
0,5659,7
1,5298,1
2,4916,6
3,4460,5
4,4182,4
5,4038,2
6,3985,3


## Exercise

In [8]:
client = bigquery.Client()

dataset_ref = client.dataset("world_bank_intl_education", project='bigquery-public-data')
dataset = client.get_dataset(dataset_ref)

table_ref = dataset_ref.table("international_education")
table = client.get_table(table_ref)

client.list_rows(table, max_results=5).to_dataframe()

,country_name,country_code,indicator_name,indicator_code,value,year
0,Nigeria,NGA,"Unemployment, male (% of male labor force)",SL.UEM.TOTL.MA.ZS,4.200000e+00,2016
1,Oman,OMN,"Population, ages 0-14, male",SP.POP.0014.MA.IN,4.954920e+05,2016
2,Pakistan,PAK,"Prevalence of HIV, total (% of population ages...",SH.DYN.AIDS.ZS,1.000000e-01,2016
3,Papua New Guinea,PNG,"GNI, PPP (current international $)",NY.GNP.MKTP.PP.CD,3.352805e+10,2016
4,Mauritania,MRT,SABER: (Engaging the Private Sector) Policy Go...,SABER.PRVT.GOAL2.LVL5,4.000000e+00,2016


In [9]:
#table.schema

In [10]:
country_spend_pct_query = """
                        SELECT country_name, AVG(value) AS avg_ed_spending_pct
                        FROM `bigquery-public-data.world_bank_intl_education.international_education`
                        WHERE indicator_code = 'SE.XPD.TOTL.GD.ZS' and year>=2010 and year<=2017
                        GROUP BY country_name
                        ORDER BY avg_ed_spending_pct DESC
                            """

In [12]:
country_spending_results = get_query(country_spend_pct_query)
print(country_spending_results.head())

            country_name  avg_ed_spending_pct
0                   Cuba            12.837270
1  Micronesia, Fed. Sts.            12.467750
2        Solomon Islands            10.001080
3                Moldova             8.372153
4                Namibia             8.349610


In [14]:
query_3 = """
        SELECT indicator_name, indicator_code, COUNT(1) AS num_rows
        FROM `bigquery-public-data.world_bank_intl_education.international_education`
        WHERE year = 2016
        GROUP BY indicator_name, indicator_code
        ORDER BY num_rows DESC
        """
query_3_result = get_query(query_3)
query_3_result.head()

,indicator_name,indicator_code,num_rows
0,Population growth (annual %),SP.POP.GROW,232
1,"Population, total",SP.POP.TOTL,232
2,Internet users (per 100 people),IT.NET.USER.P2,223
3,"Population, ages 0-14, male",SP.POP.0014.MA.IN,213
4,"Population, ages 15-64, total",SP.POP.1564.TO,213
